In [245]:
import geopandas as gpd
import pandas as pd
import glob
import os
from shapely.geometry import MultiPolygon, Polygon
from shapely.ops import unary_union

In [246]:
# 0: Geodataframe creation
def g(geom, crs):
    try:
        df = gpd.GeoDataFrame(geometry=geom, crs=crs)
        return df
    except:
        df = gpd.GeoDataFrame(geometry=[geom], crs=crs)
        return df


Experiment # 1
Interpolating points along Main County Edges to spatial join with adj. edges. Points that intersect with a single polygon SHOULD be the direct adj. poly. This poly will be used to 
merge and then triangualte. The idea behind the points is to eliminate the possibility of spatial joining 'kitty-corner' polygons which would occur if simple doing an intersect join.

Requirements:
1) Main County Edges
2) All Adj. County Edges
3) County Boundaries (County Polygons)

Steps:
1) Iterate through each Main COunty Edge and conduct the following steps
2) Interpolate points (start with 12)
3) Spatial join points df to adj. edge poly df
4) Identify adj. edge poly that occurs ONCE/Most often
5) Conduct Spatial join on Main COunty Edge poly (will get kitty-corner polys)
6) Locate the Adj. Edge Poly that will be merged
7) Take the two geometries and union (create 1 poly)
8) Trasnfer to seperate df
9) Spatial join corresponding BFEs
10) Triangulate
11) Capture Triangulated poly in single gdf (concat each iteration)
12) Split each geometry by County Polygon (Select main polygon from county shapefile)
13) Seperate triangles that are WITHIN County Polygon -- Eventually concat to Main Triangulation
                                OUTSIDE County Polygon -- Write out as (name file after County the edges are asscociated with) 

Conclusion:
Code Works! Will throw an error it seems if the Main Edge Polygon is very small and/or
there are no bfes available. Triangulation is being correctly formed regardless that 
BFEs are seperate LineStrings. As long as there are BFEs and the size of the polygon is not
too small, then the triangulation runs without a hitch. 

In [247]:
""" Read Dallas Edges  """
dal = gpd.read_file('edges_48113.shp')
dal = dal[['geometry']]
dal.reset_index(inplace=True)

dal.head()



,index,geometry
0,0,"POLYGON ((1245426.341 3677714.492, 1245427.688..."
1,1,"POLYGON ((1245624.393 3677499.866, 1245623.046..."
2,2,"POLYGON ((1245625.866 3675202.492, 1245627.835..."
3,3,"POLYGON ((1245738.083 3675188.326, 1245736.114..."
4,4,"POLYGON ((1245638.571 3675104.831, 1245639.046..."


In [248]:
""" Test on single Main Edge Poly """
test = 8
dal = g(dal.iloc[test].geometry, 26913)
dal



,geometry
0,"POLYGON ((1246547.865 3661097.078, 1246548.937..."


In [249]:
""" Read in Adj. COunty Edges (Tarrant)"""
tar = gpd.read_file('edges_48439.shp')
tar = tar[['geometry']]
tar.reset_index(inplace=True, drop=True)
tar.head()


,geometry
0,"POLYGON ((1197416.254 3674741.762, 1197416.341..."
1,"POLYGON ((1197906.981 3667938.805, 1197906.630..."
2,"POLYGON ((1198016.254 3663511.939, 1198016.339..."
3,"POLYGON ((1198021.913 3663379.458, 1198021.829..."
4,"POLYGON ((1198261.579 3663869.300, 1198260.391..."


In [250]:
""" Iterate through All Dallas Edges and do the following """
# Interpolate Points -- Convert to seperate gdf
points = 12
dal['line_geom'] = dal.apply(lambda x: x.geometry.boundary, axis=1)
for i, d in dal.iterrows():
    pts = [d.line_geom.interpolate(i/points, normalized=True) for i in range(1, points)]
    pts = g(pts, 26913)
    pts_buff = g(pts.buffer(1).geometry, 26913)

In [251]:
# Spatial join df to Adj. Edge Poly df
pjoin = pts_buff.sjoin(tar)
pjoin

,geometry,index_right
10,"POLYGON ((1246551.717 3661029.149, 1246551.712...",56


 get index that occurs most often
df = pd.DataFrame({'index': [1, 2, 2, 3, 4], 'index_right': [20, 20, 51, 20, 20]})
adjId = df['index_right'].value_counts().idxmax()

In [252]:
# Identify which index occurs only once // Most often?
adjId = pjoin['index_right'].value_counts().idxmax()
adjId

56

In [253]:
# Regular Spatial join between Dallas Edge Polys and Adj. Edge Polys
dal_join = dal.sjoin(tar)
dal_join

,geometry,line_geom,index_right
0,"POLYGON ((1246547.865 3661097.078, 1246548.937...","LINESTRING (1246547.865 3661097.078, 1246548.9...",56
0,"POLYGON ((1246547.865 3661097.078, 1246548.937...","LINESTRING (1246547.865 3661097.078, 1246548.9...",55


In [254]:
# Filter Sjoin to include only the adjID
dal_join = dal_join.loc[dal_join['index_right'] == adjId]

# Merge with Adj. COunty Edge df to get Adj. Geometry
dal_join = dal_join.merge(tar, left_on='index_right', right_index=True)
dal_join

,geometry_x,line_geom,index_right,geometry_y
0,"POLYGON ((1246547.865 3661097.078, 1246548.937...","LINESTRING (1246547.865 3661097.078, 1246548.9...",56,"POLYGON ((1246518.549 3661050.830, 1246519.592..."


In [255]:
# Union both geometries and pass to seperate gdf
dal_join['merge'] = dal_join.apply(lambda x: unary_union([x.geometry_x, x.geometry_y]), axis=1)
edge = dal_join[['merge']].rename(columns={'merge': 'geometry'})
edge = edge.set_geometry('geometry', crs=26913)
edge

,geometry
0,"POLYGON ((1246519.592 3661052.473, 1246520.635..."


In [256]:
edge.to_file(f'{test}_edge.shp')

In [257]:
""" Bfes from both counties will need to be concatenated so that they are all associated during the spatial join """
# read COunty BFEs
dal_bfe = gpd.read_file('BFE_48113.shp')
tar_bfe = gpd.read_file('BFE_48439.shp')

# spatial join to Edges polygons
dal_bfe_edge = dal_bfe.sjoin(dal)
dal_bfe_edge = dal_bfe_edge[['ELEV', 'FIPS', 'geometry']]
tar_bfe_edge = tar_bfe.sjoin(tar)
tar_bfe_edge = tar_bfe_edge[['ELEV', 'FIPS', 'geometry']]

# combine all edge BFEs
all_edge_bfe = pd.concat([dal_bfe_edge, tar_bfe_edge])
all_edge_bfe

,ELEV,FIPS,geometry
4029,476.0,48113,"LINESTRING (1246603.026 3661001.820, 1246604.1..."
4966,478.0,48113,"LINESTRING (1246546.751 3661095.337, 1246547.8..."
132,613.0,48439,"LINESTRING (1228993.358 3677351.944, 1228995.3..."
340,479.0,48439,"LINESTRING (1246498.211 3661086.410, 1246499.0..."
7589,480.0,48439,"LINESTRING (1246363.458 3661101.119, 1246365.2..."
...,...,...,...
7963,818.4,48439,"LINESTRING (1210394.584 3676078.106, 1210396.5..."
7963,818.4,48439,"LINESTRING (1210394.584 3676078.106, 1210396.5..."
8240,598.0,48439,"LINESTRING (1241891.135 3628995.315, 1241890.3..."
8241,603.0,48439,"LINESTRING (1241799.395 3628947.721, 1241797.9..."


In [258]:
# Locate corresponding BFEs to Union Edge Poly
bfe_set = all_edge_bfe.sjoin(edge)

# Why do I have duplicates in all_bfes?
bfe_set = bfe_set.drop_duplicates()
bfe_set = bfe_set[['ELEV', 'geometry']]
bfe_set

,ELEV,geometry
4029,476.0,"LINESTRING (1246603.026 3661001.820, 1246604.1..."
4966,478.0,"LINESTRING (1246546.751 3661095.337, 1246547.8..."
5541,478.0,"LINESTRING (1246517.483 3661049.153, 1246518.5..."


In [259]:
""" Begin Triangulation process on unioned edge poly using the 
corresponding BFE set. """
from new_methods import *
triangles = gpd.GeoDataFrame()

# Getting Z-geom for BFE Points
bfe_pts_utm, bfe_pts_84 = bfe_zpts(bfe_set)

# FSP Simplify, Interpolation, and Z-geom
fsp_i_pts = fsp_pts(edge, bfe_pts=bfe_pts_utm, bfe_set=bfe_set, diff_area=900)

if fsp_i_pts is not None:
    print('Creating more points..')
    fsp_i_pts = (fsp_i_pts.pipe(IDW, bfe=bfe_set, power=2)
                .pipe(ELEV_2geom)
)

    # Concat and Triangulate
    all_pts = pd.concat([bfe_pts_84, fsp_i_pts], ignore_index=True)
    all_pts_multigeom = MultiPoint(all_pts.geometry.to_list())

    tin = triangulate(all_pts_multigeom)
    tin_df = g(tin, 4326)

    # Extract Geom
    final_tin = extract_geom(tin_df)
    
    #final_tin = final_tin.overlay(f) #(final_tin.geometry.centroid.within(f.geometry[0])) |
                
    triangles = pd.concat([triangles, final_tin], ignore_index=True)

else:
    print('Using only BFE pts')
    # Concat and Triangulate
    all_pts_multigeom = MultiPoint(bfe_pts_84.geometry.to_list())

    tin = triangulate(all_pts_multigeom)
    tin_df = g(tin, 4326)

    # Extract Geom
    final_tin = extract_geom(tin_df)
    #final_tin = final_tin.overlay(f) #(final_tin.geometry.centroid.within(f.geometry[0])) |
                        
    triangles = pd.concat([triangles, final_tin], ignore_index=True)

Creating more points..


In [260]:
triangles.to_file(f'{test}_triangles.shp')